In [39]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the code."></form>
''')

<font color = 'teal' size=20>Cohort Analysis and Visualization</font><br>
Grouping customers into simple behavioral segments.<br>
This analysis can be used for two primary purposes: <br>
1. For one-time campaign retrospection
2. For ongoing user engagement benchmarking<br>

# <font color = 'teal'> Setting Up Enviornment </font>

In [1]:
# Basic data manipulation
import numpy as np
import math
import pandas as pd
pd.set_option('display.max_columns', 20)
import datetime as dt
import qgrid
import requests

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# <font color = 'teal'> Reading and Inspecting Data </font>

In [2]:
#GitHub data location
data = 'https://github.com/michael-william/Cohort-analysis/raw/master/Online%20Retail.xlsx'
source = pd.ExcelFile(data)
online = pd.read_excel(source, 'Online Retail')

Taking a look at the first few rows of data

In [3]:
#qgrid.show_grid(online, show_toolbar=True)

# or
online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


Fairly straights forward. Now taking a look at the the quality of data

In [4]:
online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


Looks like were missing some CustomerID values in some rows.
Will need to drop these rows as we need CustomerID's for this analysis

In [5]:
#Dropping nulls
online = online.dropna().reset_index(drop=True)

#Sorting to make it easier to spot check
online = online.sort_values('CustomerID').reset_index(drop=True)

### Assign daily acquisition cohort (CohortDay)##
Creating a CohortDay for each Customer telling us when they made their first purchase.

In [6]:
# Define a function that will parse the date
def get_day(x): 
    return dt.datetime(x.year, x.month, x.day) 

# Create InvoiceDay column
online['InvoiceDay'] = online['InvoiceDate'].apply(get_day) 

# Group by CustomerID and select the InvoiceDay value
grouping = online.groupby('CustomerID')['InvoiceDay'] 

# Assign a minimum InvoiceDay value to the dataset
online['CohortDay'] = grouping.transform('min').astype('datetime64[ns]')

# View the top 5 rows
online.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,CohortDay
0,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,2011-01-18,2011-01-18
1,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,2011-01-18,2011-01-18
2,562032,21578,WOODLAND DESIGN COTTON TOTE BAG,6,2011-08-02 08:48:00,2.25,12347.0,Iceland,2011-08-02,2010-12-07
3,542237,47559B,TEA TIME OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,2011-01-26,2010-12-07
4,542237,21154,RED RETROSPOT OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,2011-01-26,2010-12-07


### Calculate time offset in days - part 1##
Creating lists of values showing the invoice year, month, and day
Also creating lists of cohort year, month, and day

In [7]:
def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

# Get the integers for date parts from the `InvoiceDay` column
invoice_year, invoice_month, invoice_day = get_date_int(online, 'InvoiceDay')

# Get the integers for date parts from the `CohortDay` column
cohort_year, cohort_month, cohort_day = get_date_int(online, 'CohortDay')

### Calculate time offset in days - part 2 (ChortIndex)
Getting the difference in years, months, and days between each invoice and the cohort date for each customer.
This column will be called the Cohort Index and will be measured in months.

In [8]:
# Calculate difference in years
years_diff = invoice_year - cohort_year

# Calculate difference in months
months_diff = invoice_month - cohort_month

# Calculate difference in days
days_diff = invoice_day - cohort_day

# Extract the difference in days from all previous values
online['CohortIndex'] = years_diff * 365 + months_diff * 30 + days_diff + 1

# Converting CohortIndex to months from days
online['CohortIndex'] = np.ceil(np.divide(online.CohortIndex,30)).astype('int')

online.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,InvoiceDay,CohortDay,CohortIndex
0,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,2011-01-18,2011-01-18,1
1,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,2011-01-18,2011-01-18,1
2,562032,21578,WOODLAND DESIGN COTTON TOTE BAG,6,2011-08-02 08:48:00,2.25,12347.0,Iceland,2011-08-02,2010-12-07,9
3,542237,47559B,TEA TIME OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,2011-01-26,2010-12-07,2
4,542237,21154,RED RETROSPOT OVEN GLOVE,10,2011-01-26 14:30:00,1.25,12347.0,Iceland,2011-01-26,2010-12-07,2


# <font color = 'teal'> Cohort Metrics </font>
First, we'll look at retention of customers by their CohortMonth. This is a basic calculation that looks at the unique number of customers who made a purchase in the months after their first purchase.

### Calculate retention from scratch

In [9]:
#Adding CohortMonth from CohortDay in dt format
online['CohortMonth'] =  online.CohortDay + pd.offsets.MonthBegin(1)

#Grouping by month and index and counting unique customer ids
grouping = online.groupby(['CohortMonth', 'CohortIndex'])
cohort_data = grouping['CustomerID'].apply(pd.Series.nunique).reset_index()

# Create a pivot 
cohort_counts = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='CustomerID')

# Select the first column and store it to cohort_sizes
cohort_sizes = cohort_counts.iloc[:,0]

# Divide the cohort count by cohort sizes along the rows
retention = (cohort_counts.divide(cohort_sizes, axis=0) * 100).round(2)
retention.head()

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2011-01-01,100.0,38.29,34.49,38.50,36.29,39.03,37.03,35.44,34.92,38.29,40.61,50.74,19.73
2011-02-01,100.0,24.70,28.98,30.40,30.17,28.98,25.65,27.55,33.73,35.15,28.50,1.43,NaN
2011-03-01,100.0,23.42,23.95,29.47,20.79,26.84,25.53,27.89,28.95,25.00,0.79,NaN,NaN
2011-04-01,100.0,18.64,25.00,23.18,19.55,22.05,24.32,27.05,21.82,2.50,NaN,NaN,NaN
2011-05-01,100.0,27.42,17.73,22.41,18.39,23.75,24.75,22.74,2.34,NaN,NaN,NaN,NaN


### Calculate average price
Now we'll do a similar analysis and determine the average amount spent each month.

In [10]:
# Create a groupby object and pass the monthly cohort and cohort index as a list
grouping = online.groupby(['CohortMonth', 'CohortIndex']) 

# Calculate the average of the unit price column
cohort_data = grouping['UnitPrice'].mean()

# Reset the index of cohort_data
cohort_data = cohort_data.reset_index()

# Create a pivot 
average_price = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='UnitPrice')
average_price = average_price.round(2)
average_price.head()

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2011-01-01,3.20,3.19,3.23,3.57,2.90,4.99,3.15,3.25,3.45,3.08,3.21,2.85,2.81
2011-02-01,3.50,3.38,7.34,3.12,3.33,2.88,2.77,2.72,5.63,3.00,2.77,2.36,NaN
2011-03-01,3.52,4.78,2.94,3.14,2.97,2.72,3.19,3.00,2.89,3.06,2.54,NaN,NaN
2011-04-01,3.49,3.86,3.32,3.53,3.08,2.79,2.77,2.79,2.66,3.15,NaN,NaN,NaN
2011-05-01,3.42,3.82,3.16,2.86,2.88,2.88,2.72,2.91,2.54,NaN,NaN,NaN,NaN


### Calculate average quantity

In [11]:
# Create a groupby object and pass the monthly cohort and cohort index as a list
grouping = online.groupby(['CohortMonth', 'CohortIndex']) 

# Calculate the average of the unit price column
cohort_data = grouping['Quantity'].mean()

# Reset the index of cohort_data
cohort_data = cohort_data.reset_index()

# Create a pivot 
average_quantity = cohort_data.pivot(index='CohortMonth', columns='CohortIndex', values='Quantity')
average_quantity = average_quantity.round(1)
average_quantity.head()

CohortIndex,1,2,3,4,5,6,7,8,9,10,11,12,13
CohortMonth,,,,,,,,,,,,,
2011-01-01,11.3,14.5,14.1,15.0,14.6,13.3,14.4,17.5,14.8,17.2,16.9,13.0,13.1
2011-02-01,9.8,15.2,11.3,10.7,12.7,15.4,14.2,14.0,11.4,8.5,9.2,12.2,NaN
2011-03-01,10.8,13.9,16.1,11.8,10.4,13.2,13.0,11.2,10.0,13.8,18.8,NaN,NaN
2011-04-01,9.7,11.0,11.8,9.9,13.8,13.9,10.8,13.7,9.3,5.7,NaN,NaN,NaN
2011-05-01,9.8,8.7,11.8,11.3,11.1,8.6,9.3,9.5,5.6,NaN,NaN,NaN,NaN


# <font color = 'teal'>Cohot Analysis Visualization</font>

Visualizing retention by CohortMonth.<br>
Customers who made their first purchase in Jan 2011 seemed to have the best retention.<br>
The 2nd month of any CohortMonth seems to have a bigger drop in retention than other months.<br>
Also, the last month of any cohort is extremely low indicating partial monthly data.

In [41]:
r_heat = retention.drop(1,axis=1)[:-1]
fig = go.Figure(data=go.Heatmap(
                   z=r_heat.values,
                   x=r_heat.columns,
                   y=r_heat.index,
                   hovertemplate ='<i>CohortMonth</i>: %{y}<br>'+'<i>Retention</i>: %{z}'+'%'+'<extra></extra>',
                   #text = ['Custom text {}'.format(r_heat.values)],
                   hoverongaps = False))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title='Retention by CohortMonth', 
                  xaxis={'title':'Following months', 'tickmode':'linear'}, 
                  yaxis={'title':'CohortMonth', 'tickangle':0, 'automargin': True})
fig.show()

Visualizing average price by CohortMonth.<br>
Customers who made their first purchase in Sept 2011 seemed to have the highest average price spent.<br>
The second month of any cohort seems better than the other following months.

In [43]:
p_heat = average_price.drop(1,axis=1)[:-1]
fig = go.Figure(data=go.Heatmap(
                   z=p_heat.values,
                   x=p_heat.columns,
                   y=p_heat.index,
                   colorscale="Viridis", 
                   hovertemplate ='<i>CohortMonth</i>: %{y}<br>'+'<i>Average price</i>: $%{z}<extra></extra>',
                   hoverongaps = False))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title='Average Price by CohortMonth', 
                  xaxis={'title':'Following months', 'tickmode':'linear'}, 
                  yaxis={'title':'CohortMonth', 'tickangle':0, 'automargin': True})
fig.show()

Visualizing average quantity by CohortMonth.<br>
Interesting that the average quantity for Sept 2011 is low when the average price spend is high<br>
This indicates that the customers are buying higher priced items during this time.

In [45]:
q_heat = average_quantity.drop(1,axis=1)[:-1]
fig = go.Figure(data=go.Heatmap(
                   z=q_heat.values,
                   x=q_heat.columns,
                   y=q_heat.index,
                   colorscale="cividis",
                   hovertemplate ='<i>CohortMonth</i>: %{y}<br>'+'<i>Average quantity</i>: %{z} items<extra></extra>',
                   hoverongaps = False))
fig.update_yaxes(autorange="reversed")
fig.update_layout(title='Average Quantity by CohortMonth', 
                  xaxis={'title':'Following months', 'tickmode':'linear'}, 
                  yaxis={'title':'CohortMonth', 'tickangle':0, 'automargin': True})
fig.show()

In [44]:
fig.write_html("/Users/michaelcondon/Documents/GitHub/Cohort-analysis/v_heatmap.html")

These traditional Cohort visualizations are nice and can start some good conversation, but I find behavioral segmentation to have more actionable insight.

# <font color = 'teal'>RFMT Sementation Recency Frequency Monetary</font>
https://en.wikipedia.org/wiki/RFM_(market_research)
- Recency - days since last customer transaction
- Frequecy - number of transaction in the last 12 months
- Monetary Value - total spend in last 12 months
- Tenure - days since first transaction

**Can group by:**
  - Percentiles (chosen for this project)
  > 1. Sort customers based on percentile
  > 2. Break customers into a pre-defined number of groups of equal size
  > 3. Assign a label to each group
  - Pareto 80/20 cut
  - Custom, based on business knowledge


### Adjusting data to only last 12 months of invoices

In [15]:
print('Min invoice date:{}; Max invoice data:{}'.format(min(online.InvoiceDate), max(online.InvoiceDate)))

Min invoice date:2010-12-01 08:26:00; Max invoice data:2011-12-09 12:50:00


Data is old, so creating hypothetical snap_shot date 

In [16]:
#Data is already adjusted to last 12 months 
#Data is old, so creating hypothetical snap_shot data as if we're doing analysis recently
snapshot_date = max(online.InvoiceDate) + dt.timedelta(days=1)

### Calculate RFMT Metrics<br>
* Recency - days since last purchase
* Frequency - number of purchases
* Monetary Values - amount spent
* Tenure - days since first purchase

Aggregating data and adding a column showing the amount of days since last purchase

In [17]:
# Aggregating data on a customer level
rfmt = online.groupby('CustomerID')
rfmt = rfmt.agg(Recency = ('InvoiceDate','max'),
            Frequency = ('InvoiceNo','size'),
           Monetary_Value = ('UnitPrice', 'sum'),
             Tenure = ('InvoiceDate','min'))

#Creating Recency column
recency_days = []
for x in rfmt.Recency:
    diff = snapshot_date-x
    diff = diff/np.timedelta64(1,"D")
    diff = np.ceil(diff).astype('int')
    recency_days.append(diff)

rfmt['Recency'] = recency_days

#Creating Tenure column
tenure_days = []
for x in rfmt.Tenure:
    diff = snapshot_date-x
    diff = diff/np.timedelta64(1,"D")
    diff = np.ceil(diff).astype('int')
    tenure_days.append(diff)

rfmt['Tenure'] = tenure_days

#Setting index to CustomerID
rfmt.index = rfmt.index.astype('int')
rfmt.index.name='CustomerID'

# Dropping rows where monetary value = 0 for later pre-processing steps
rfmt = rfmt[rfmt.Monetary_Value != 0]

rfmt.head()

,Recency,Frequency,Monetary_Value,Tenure
CustomerID,,,,
12346,327,2,2.08,327
12347,3,182,481.21,368
12348,76,31,178.71,359
12349,20,73,605.10,20
12350,311,17,65.30,311


### Building RFMT segments<br>
Assinging each customer a quartile value for each column where the top quartile is 4 and the bottom is 1.<br>
We will then combine these quartiles to create the segment<br>
Segments will be summed to get a segment score

In [18]:
import warnings
warnings.filterwarnings('ignore')

# Generator of labels for recency where 4 is best and 1 is worst
r_labels = range(4,0,-1)

# Recency quartiles
r_quartiles = pd.qcut(rfmt.Recency,4,labels=r_labels)

# Append recency quartiles to dataframe
rfmt['R'] = r_quartiles

# Generator of labels for frequency and monetary where 1 is best and 4 is worst
f_labels = range(1,5)
m_labels = range(1,5)
t_labels = range(1,5)

# frequency and monetary quartiles
f_quartiles = pd.qcut(rfmt.Frequency,4,labels=f_labels)
m_quartiles = pd.qcut(rfmt.Monetary_Value,4,labels=m_labels)
t_quartiles = pd.qcut(rfmt.Tenure,4,labels=t_labels)

# Append recency quartiles to dataframe
rfmt['F'] = f_quartiles
rfmt['M'] = m_quartiles
rfmt['T'] = t_quartiles

# Creating RFMT segement
def join_rfmt(x):
    return str(x.R) + str(x.F) + str(x.M) + str(x['T'])

rfmt['Segment'] = rfmt.apply(join_rfmt,axis=1)

rfmt['Score'] = rfmt[['R','F','M','T']].sum(axis=1).astype('int')

rfmt.head()

,Recency,Frequency,Monetary_Value,Tenure,R,F,M,T,Segment,Score
CustomerID,,,,,,,,,,
12346,327,2,2.08,327,1,1,1,3,1113,6
12347,3,182,481.21,368,4,4,4,4,4444,16
12348,76,31,178.71,359,2,2,3,4,2234,11
12349,20,73,605.10,20,3,3,4,1,3341,11
12350,311,17,65.30,311,1,1,2,3,1123,7


### Analyzinf RFMT Segments

The '4444' segment is our most desirable as they are in the top quartiles of each column.<br>
The '1112' segemnt is our least desirable segment.<br>
The good news is that the '4444' segment has the highest share of all the segments.<br>
However, this is a bit hard to look at with all the different combinations of segments equaling the same values.

In [19]:
# Calculate size of segments
s_size = rfmt.groupby('Segment').size().sort_values(ascending=False).reset_index()
s_size.columns = ['Segment','Count']
fig = px.pie(s_size,values='Count', names='Segment', title='Segement Size')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

The visualization of 'Score' is much better as there are less unique values.
See how the insights have changed with a score of 7 being the most frequent.

In [20]:
# Calculate size of segments
score_size = rfmt.groupby('Score').size().sort_values(ascending=False).reset_index()
score_size.columns = ['Score','Count']
fig = px.pie(score_size,values='Count', names='Score', title='Score Size')
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

Well now divide the 'Score' column into three General Segments:
* Top = A score of 13 and higher
* Middle = A score greater than 8 and lower than 13
* Bottom = A score equal-to or lower-than 8

In [21]:
# Summary metrics for RFMT Score
rfmt.groupby('Score').agg(Recency_avg = ('Recency','mean'),
                         Frequency_avg = ('Frequency','mean'), 
                         Monetary_avg =('Monetary_Value','mean'),
                         Tenure =('Tenure','mean'))

# function for grouping into named segemnts by RFM Score
def segment_me(df):
    if df['Score'] >= 13:
        return 'Top'
    elif df['Score'] >=9 and (df['Score'] < 13):
        return 'Middle'
    else:
        return 'Bottom'

rfmt['General_Segment'] = rfmt.apply(segment_me, axis=1)
rfmt.head()

,Recency,Frequency,Monetary_Value,Tenure,R,F,M,T,Segment,Score,General_Segment
CustomerID,,,,,,,,,,,
12346,327,2,2.08,327,1,1,1,3,1113,6,Bottom
12347,3,182,481.21,368,4,4,4,4,4444,16,Top
12348,76,31,178.71,359,2,2,3,4,2234,11,Middle
12349,20,73,605.10,20,3,3,4,1,3341,11,Middle
12350,311,17,65.30,311,1,1,2,3,1123,7,Bottom


We can now group by the General Segment and get the averages for RFMT

In [22]:
segments = rfmt.groupby('General_Segment').agg(Size = ('General_Segment','count'),
                                   Recency_avg = ('Recency','mean'), 
                                   Frequency_avg = ('Frequency','mean'), 
                                   Monetary_avg = ('Monetary_Value', 'mean'),
                                   Tenure_avg = ('Tenure','mean')).round(1).reindex(['Top', 'Middle', 'Bottom'])

segments['Share_Percent'] = (segments.Size/np.sum(segments.Size)*100).round(2)
segments = segments[['Size', 'Share_Percent', 'Recency_avg', 'Frequency_avg', 'Monetary_avg' ]]

segments

,Size,Share_Percent,Recency_avg,Frequency_avg,Monetary_avg
General_Segment,,,,,
Top,1153,26.38,25.6,249.3,849.1
Middle,1516,34.68,82.8,60.7,193.7
Bottom,1702,38.94,147.9,16.1,79.4


# <font color='teal'> Data Pre-Processing for k-means Clustering for Unsupervised Learning</font>

### Assumptions about data for k-means##
<br>
 1. Symetric distribution of variables (not skewed)<br>
 2. All variables have same average values to get equal weight in k-means calculation<br>
 3. All variables have the same variance <br> 

### 1. Managing skewed variables

In [23]:
#Creating rfmt_data for preprocessing
rfmt_data = rfmt[['Recency', 'Frequency','Monetary_Value', 'Tenure']]

#Unskew the data
rfmt_log = np.log(rfmt_data)

#Initialize a standard scaler and fit it
scaler = StandardScaler()
scaler.fit(rfmt_log)

#Scale and center the data
rfmt_normalized = scaler.transform(rfmt_log)

#Create a pandas DataFrame
rfmt_normalized = pd.DataFrame(data=rfmt_normalized, index=rfmt_data.index, columns=rfmt_data.columns)

In [24]:
fig = make_subplots(rows=1, cols=4)

trace1 = go.Histogram(x=list(rfmt_normalized['Recency'].values), nbinsx=10, name='Recency', opacity=.7
)

trace2 = go.Histogram(x=list(rfmt_normalized['Frequency'].values), nbinsx=10, name='Frequency', opacity=.7
)

trace3 = go.Histogram(x=list(rfmt_normalized['Monetary_Value'].values), nbinsx=10, name='Monetary Value', opacity=.7
)

trace4 = go.Histogram(x=list(rfmt_normalized['Tenure'].values), nbinsx=10, name='Tenure', opacity=.7
)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)
fig.append_trace(trace3, 1, 3)
fig.append_trace(trace4, 1, 4)

fig.update_layout(title_text="Distribtuion of Values after Nomalization", xaxis={'title':'Value'}, yaxis={'title':'Frequency'})
fig.show()

In [25]:
# Converting to dataframe 
rfmt_normalized= pd.DataFrame(rfmt_normalized,index = rfmt_log.index, columns=rfmt_log.columns)
rfmt_normalized.describe().round(2)

,Recency,Frequency,Monetary_Value,Tenure
count,4371.00,4371.00,4371.00,4371.00
mean,0.00,-0.00,-0.00,-0.00
std,1.00,1.00,1.00,1.00
min,-2.78,-2.75,-4.65,-5.07
25%,-0.66,-0.64,-0.58,-0.46
50%,0.10,0.04,0.05,0.42
75%,0.86,0.70,0.65,0.73
max,1.56,3.96,4.13,0.86


### Choose a number of clusters with k-means
***Methods for defining number of clusters***<br>
- Visual methods: elbow criterion (plotting # of clusters against sum-of-squared errors)<br>
- Mathmatical methods: silouette coeffcient<br>
- Experimentation and iterperation<br>
<br>
We will be using the Elbow Method to interate through clusters ranging from 1-10. The ideal cluster range should be where we see the biggest change in slope. Here we can see that the ideal range is around 3 to 4 clusters.


In [26]:
### Elbow criterion ### 

#Fit KMeans and calculate SSE for each k
sse = {}

#Initialize kmeans with k
for k in range(1,11):
    kmeans = KMeans(n_clusters=k, random_state=1)
    kmeans.fit(rfmt_normalized)
    sse[k] = kmeans.inertia_ # sum of squared distances to closest cluster center

fig = go.Figure(data=go.Scatter(x=list(sse.keys()), y=list(sse.values()), mode='markers+lines'
))

fig.update_layout(title='Elbow Method for Clusters', xaxis={'title':'Number of clusters'}, yaxis={'title':'Sum squared distances'})
fig.show()

Using KMeans, we will set the number of clusters to 3 and create a new data frame with averages of each column. This is similar to what we did with quartiles, but with more precision than just using quartiles.

In [27]:
### Manual cluster selection ### 
#Initialize kmeans
kmeans = KMeans(n_clusters=3, random_state=1)

# Compute k-means clustering on preprocessed data
kmeans.fit(rfmt_normalized)

# Extract the cluster labels
cluster_labels = kmeans.labels_
rfmt_k3 = rfmt.assign(Cluster = cluster_labels)

# Analyze result
rfmt_3group = rfmt_k3.groupby(['Cluster']).agg(Size = ('General_Segment','count'),
                    Recency = ('Recency','mean'), 
                              Frequency = ('Frequency','mean'), 
                              Monetary_Value = ('Monetary_Value','mean'),
                                     Tenure=('Tenure','mean')).round(2)
rfmt_3group

,Size,Recency,Frequency,Monetary_Value,Tenure
Cluster,,,,,
0,1799,34.65,188.06,659.11,285.73
1,1657,187.97,23.49,84.40,258.91
2,915,36.03,32.33,89.86,52.90


We will also create a KMeans model with 4 clusters to compare to the 3 cluster model

In [28]:
# Create a cluster label column in the original df with kmeans solution
# KMeans elbow shows '3'or'4' as the best number of clusters
kmeans4 = KMeans(n_clusters=4, random_state=1)

# Compute k-means clustering on preprocessed data
kmeans4.fit(rfmt_normalized)

# Extract the cluster labels
cluster_labels = kmeans4.labels_
rfmt_k4 = rfmt.assign(Cluster = cluster_labels)

rfmt_4group = rfmt_k4.groupby(['Cluster']).agg(Size = ('General_Segment','count'),
                    Recency = ('Recency','mean'), 
                              Frequency = ('Frequency','mean'), 
                              Monetary_Value = ('Monetary_Value','mean'),
                                     Tenure=('Tenure','mean')).round(2)
rfmt_4group


,Size,Recency,Frequency,Monetary_Value,Tenure
Cluster,,,,,
0,851,33.15,35.10,99.74,49.51
1,1501,136.81,51.37,203.24,264.47
2,760,197.80,8.53,26.67,237.46
3,1259,18.17,233.02,792.38,295.38


### Profile and interpret segments

**Aproaches to build customer personas**<br>
- Summary statistics for each cluster e.g. average RFM values
- Snake plots (from market reserach)
- Calculate relative importance of cluster attributes compared to population

Stacking the data to prepare it for visualization

In [38]:
rfmt_3stack = rfmt_3group.drop('Size', axis=1).stack().reset_index()
rfmt_3stack.columns=['Cluster', 'Attribute', 'Average']
k3_new_labels = {1:'Sleepers',2:'New',0:'Loyal'}
rfmt_3stack = rfmt_3stack.replace({'Cluster':k3_new_labels})
rfmt_3stack

,Cluster,Attribute,Average
0,Loyal,Recency,34.65
1,Loyal,Frequency,188.06
2,Loyal,Monetary_Value,659.11
3,Loyal,Tenure,285.73
4,Sleepers,Recency,187.97
5,Sleepers,Frequency,23.49
6,Sleepers,Monetary_Value,84.40
7,Sleepers,Tenure,258.91
8,New,Recency,36.03
9,New,Frequency,32.33


Here we can see the average values for each attribute:
* Recency - Average # of days since last purchase per customer (lower is better)
* Frequency - Average number of purchases per customer (high is better)
* Monetary Value - Average amount of money spent per customer (higher is better)
* Tenure - Average # of days since the first purchase per customer (high is better)

In [31]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=rfmt_3stack[rfmt_3stack["Cluster"]=='Loyal']['Attribute'], y=rfmt_3stack[rfmt_3stack["Cluster"]=='Loyal']["Average"],
                    mode='lines+markers',
                    opacity=.7,
                    line={'color':'seagreen', 'width':3},
                    hovertemplate ='<i>Cluster: Loyal</i><br>'+'<i>Average value</i>: %{y}<extra></extra>',
                    name='Loyal'))
fig.add_trace(go.Scatter(x=rfmt_3stack[rfmt_3stack["Cluster"]=='Sleepers']['Attribute'], y=rfmt_3stack[rfmt_3stack["Cluster"]=='Sleepers']["Average"],
                    mode='lines+markers',
                    opacity=.7,
                    line={'color':'mediumslateblue', 'width':3},
                    hovertemplate ='<i>Cluster: Sleepers</i><br>'+'<i>Average value</i>: %{y}<extra></extra>',
                    name='Sleepers'))
fig.add_trace(go.Scatter(x=rfmt_3stack[rfmt_3stack["Cluster"]=='New']['Attribute'], y=rfmt_3stack[rfmt_3stack["Cluster"]=='New']["Average"],
                    mode='lines+markers',
                    opacity=.7,
                    line={'color':'darksalmon', 'width':3},
                    hovertemplate ='<i>Cluster: New</i><br>'+'<i>Average value</i>: %{y} <extra></extra>',
                    name='New'))

fig.update_layout(title='Average Values for Clusters of 3', xaxis={'title':'Attribute'}, yaxis={'title':'Average value'}, hovermode="x")

fig.show()

See how we can give personas to each cluster based on their behavior.<br> Of course, Personas may change based on the number of clusters.

In [32]:
rfmt_4stack = rfmt_4group.drop('Size', axis=1).stack().reset_index()
rfmt_4stack.columns=['Cluster', 'Attribute', 'Average']
k4_new_labels = {3:'New',1:'Gold',2:'Silver',0:'Bronze'}
rfmt_4stack = rfmt_4stack.replace({'Cluster':k4_new_labels})
rfmt_4stack

,Cluster,Attribute,Average
0,Bronze,Recency,33.15
1,Bronze,Frequency,35.10
2,Bronze,Monetary_Value,99.74
3,Bronze,Tenure,49.51
4,Gold,Recency,136.81
5,Gold,Frequency,51.37
6,Gold,Monetary_Value,203.24
7,Gold,Tenure,264.47
8,Silver,Recency,197.80
9,Silver,Frequency,8.53


Here we can see different labeling of personas with the edition of a fourth cluster.<br>
i thought it made more sense to look at clusters in a traditional segmentation

In [33]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=rfmt_4stack[rfmt_4stack["Cluster"]=='Gold']['Attribute'], y=rfmt_4stack[rfmt_4stack["Cluster"]=='Gold']["Average"],
                    mode='lines+markers',
                    opacity=.7,
                    line={'color':'seagreen', 'width':3},
                    hovertemplate ='<i>Cluster: Gold</i><br>'+'<i>Average value</i>: %{y}<extra></extra>',
                    name='Gold'))
fig.add_trace(go.Scatter(x=rfmt_4stack[rfmt_4stack["Cluster"]=='Silver']['Attribute'], y=rfmt_4stack[rfmt_4stack["Cluster"]=='Silver']["Average"],
                    mode='lines+markers',
                    opacity=.7,
                    line={'color':'mediumslateblue', 'width':3},
                    hovertemplate ='<i>Cluster: Silver</i><br>'+'<i>Average value</i>: %{y}<extra></extra>',
                    name='Silver'))
fig.add_trace(go.Scatter(x=rfmt_4stack[rfmt_4stack["Cluster"]=='Bronze']['Attribute'], y=rfmt_4stack[rfmt_4stack["Cluster"]=='Bronze']["Average"],
                    mode='lines+markers',
                    opacity=.7,
                    line={'color':'darksalmon', 'width':3},
                    hovertemplate ='<i>Cluster: Bronze</i><br>'+'<i>Average value</i>: %{y}<extra></extra>',
                    name='Bronze'))
fig.add_trace(go.Scatter(x=rfmt_4stack[rfmt_4stack["Cluster"]=='New']['Attribute'], y=rfmt_4stack[rfmt_4stack["Cluster"]=='New']["Average"],
                    mode='lines+markers',
                    opacity=.7,
                    line={'color':'purple', 'width':3},
                    hovertemplate ='<i>Cluster: New</i><br>'+'<i>Average value</i>: %{y}<extra></extra>',
                    name='New'))

fig.update_layout(title='Average Values for Clusters of 4', xaxis={'title':'Attribute'}, yaxis={'title':'Average value'}, hovermode='x')

fig.show()

# <font color='teal'> Conclusion</font><br>
We can see how an RFMT analysis can give us better insight into our customers vs the traditional cohort analysis. We used the Elbow method and KMeans to help us decide the optimum number of clusters and then transformed our data to be visualized. The next steps would be to discuss with the team on the right number of clusters for our business and decide the proper strategy to take for each cluster.